In [1]:
output_file_location = "/users/aricrowe/localscratch/output-test.hdf5"

In [2]:
from contra import ParticleType, ArrayReorder
from contra.io import SnapshotEAGLE, LineOfSightFileEAGLE
import h5py as h5
import numpy as np
from typing import Dict, Tuple
import os


/users/aricrowe/.local/share/hatch/env/virtual/contra/eXXHy1rr/contra/lib/python3.10/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


In [3]:
snapshots: Dict[int, SnapshotEAGLE] = {}
snapshots[396] = SnapshotEAGLE("/mnt/archive/projects/EAGLE/L0012N0188/REFERENCE/data/snipshot_396_z000p012/snip_396_z000p012.0.hdf5")
snapshots[397] = SnapshotEAGLE("/mnt/archive/projects/EAGLE/L0012N0188/REFERENCE/data/snipshot_397_z000p008/snip_397_z000p008.0.hdf5")

snap_nums = list(snapshots.keys())
snap_nums.sort()

snap_redshifts = { n : snapshots[n].z for n in snap_nums}
snap_expansion_factors = { n : snapshots[n].a for n in snap_nums}

def find_neighbouring_snapshots(z: float) -> Tuple[int, int]:
    if z > snapshots[snap_nums[0]].z or z < snapshots[snap_nums[-1]].z:
        raise ValueError(f"Redshift {z} outside of redshift range of avalible data.")
    lower_redshift_snap_num = snap_nums[0]
    while snapshots[lower_redshift_snap_num].z > z:
        lower_redshift_snap_num += 1
    return (lower_redshift_snap_num - 1, lower_redshift_snap_num)


In [4]:
#raw_ids = { n : snapshots[n].get_IDs(ParticleType.gas) for n in snap_nums}
#matching_order = { snap_nums[i] : ArrayReorder.create(raw_ids[snap_nums[i]], raw_ids[snap_nums[i + 1]]) for i in range(len(snap_nums) - 1) }


In [5]:
#raw_positions    = { n : snapshots[n].get_positions(ParticleType.gas).to("Mpc")       for n in snap_nums }
#raw_positions_x  = { n : raw_positions[n][:, 0]                                       for n in snap_nums }
#raw_positions_y  = { n : raw_positions[n][:, 1]                                       for n in snap_nums }
#raw_positions_z  = { n : raw_positions[n][:, 2]                                       for n in snap_nums }
#raw_masses       = { n : snapshots[n].get_masses(ParticleType.gas).to("Msun")         for n in snap_nums }
#raw_metalicities = { n : snapshots[n].get_metalicities(ParticleType.gas)              for n in snap_nums }
#raw_temperatures = { n : snapshots[n].get_temperatures(ParticleType.gas).to("K")      for n in snap_nums }
#raw_densities    = { n : snapshots[n].get_densities(ParticleType.gas).to("g/(cm**3)") for n in snap_nums }


In [6]:
#matched_positions_x  = { n : matching_order[n](raw_positions_x [n]) for n in snap_nums[:-1] }
#matched_positions_y  = { n : matching_order[n](raw_positions_y [n]) for n in snap_nums[:-1] }
#matched_positions_z  = { n : matching_order[n](raw_positions_z [n]) for n in snap_nums[:-1] }
#matched_masses       = { n : matching_order[n](raw_masses      [n]) for n in snap_nums[:-1] }
#matched_metalicities = { n : matching_order[n](raw_metalicities[n]) for n in snap_nums[:-1] }
#matched_temperatures = { n : matching_order[n](raw_temperatures[n]) for n in snap_nums[:-1] }
#matched_densities    = { n : matching_order[n](raw_densities   [n]) for n in snap_nums[:-1] }


In [7]:
los_files = ["/mnt/archive/projects/EAGLE/L0012N0188/REFERENCE/data/los/part_los_z0.010.hdf5"]


In [8]:
element_weightings = np.array([
    1.0, # X-position
    1.0, # Y-position
    1.0, # Z-position
    1.0, # Mass
    1.0, # Metalicity
    1.0, # Temperature
    1.0, # Density
])

# Create output file
if not os.path.exists(output_file_location):
    h5.File(output_file_location, "w").close()
    complete_files = []
else:
    with h5.File(output_file_location, "r") as file:
        complete_files = list(file)

for f in los_files:
    sightline_file = LineOfSightFileEAGLE(f)

    output_file_group_name = f.rsplit(os.path.sep, maxsplit = 1)[-1]

    completed_sightlines = 0
    if output_file_group_name in complete_files:
        with h5.File(output_file_location, "r") as file:
            completed_sightlines = len(list(file[output_file_group_name]))
        if completed_sightlines == len(sightline_file):
            continue
    else:
        with h5.File(output_file_location, "a") as file:
            file.create_group(output_file_group_name)

    print(output_file_group_name)

    snap_num_initial, snap_num_final = find_neighbouring_snapshots(sightline_file.z)

    selected_snap_nums = [snap_num_initial, snap_num_final]

    # Snipshot data
    raw_ids = { n : snapshots[n].get_IDs(ParticleType.gas) for n in selected_snap_nums}
    matching_order = { selected_snap_nums[i] : ArrayReorder.create(raw_ids[selected_snap_nums[i]], raw_ids[selected_snap_nums[i + 1]]) for i in range(len(selected_snap_nums) - 1) }

    raw_positions    = { n : snapshots[n].get_positions(ParticleType.gas).to("Mpc")       for n in selected_snap_nums }
    raw_positions_x  = { n : raw_positions[n][:, 0]                                       for n in selected_snap_nums }
    raw_positions_y  = { n : raw_positions[n][:, 1]                                       for n in selected_snap_nums }
    raw_positions_z  = { n : raw_positions[n][:, 2]                                       for n in selected_snap_nums }
    raw_masses       = { n : snapshots[n].get_masses(ParticleType.gas).to("Msun")         for n in selected_snap_nums }
    raw_metalicities = { n : snapshots[n].get_metalicities(ParticleType.gas)              for n in selected_snap_nums }
    raw_temperatures = { n : snapshots[n].get_temperatures(ParticleType.gas).to("K")      for n in selected_snap_nums }
    raw_densities    = { n : snapshots[n].get_densities(ParticleType.gas).to("g/(cm**3)") for n in selected_snap_nums }

    matched_positions_x  = { n : matching_order[n](raw_positions_x [n]) for n in selected_snap_nums[:-1] }
    matched_positions_y  = { n : matching_order[n](raw_positions_y [n]) for n in selected_snap_nums[:-1] }
    matched_positions_z  = { n : matching_order[n](raw_positions_z [n]) for n in selected_snap_nums[:-1] }
    matched_masses       = { n : matching_order[n](raw_masses      [n]) for n in selected_snap_nums[:-1] }
    matched_metalicities = { n : matching_order[n](raw_metalicities[n]) for n in selected_snap_nums[:-1] }
    matched_temperatures = { n : matching_order[n](raw_temperatures[n]) for n in selected_snap_nums[:-1] }
    matched_densities    = { n : matching_order[n](raw_densities   [n]) for n in selected_snap_nums[:-1] }
    # End Snipshot data

    interp_fraction = (sightline_file.a - snap_expansion_factors[snap_num_initial]) / (snap_expansion_factors[snap_num_final] - snap_expansion_factors[snap_num_initial])
    #TODO: apply box wrapping?
    interpolated_positions_x  = matched_positions_x [snap_num_initial] * (1 - interp_fraction) + (raw_positions_x [snap_num_final] * interp_fraction)
    interpolated_positions_y  = matched_positions_y [snap_num_initial] * (1 - interp_fraction) + (raw_positions_y [snap_num_final] * interp_fraction)
    interpolated_positions_z  = matched_positions_z [snap_num_initial] * (1 - interp_fraction) + (raw_positions_z [snap_num_final] * interp_fraction)
    interpolated_masses       = matched_masses      [snap_num_initial] * (1 - interp_fraction) + (raw_masses      [snap_num_final] * interp_fraction)
    interpolated_metalicities = matched_metalicities[snap_num_initial] * (1 - interp_fraction) + (raw_metalicities[snap_num_final] * interp_fraction)
    interpolated_temperatures = matched_temperatures[snap_num_initial] * (1 - interp_fraction) + (raw_temperatures[snap_num_final] * interp_fraction)
    interpolated_densities    = matched_densities   [snap_num_initial] * (1 - interp_fraction) + (raw_densities   [snap_num_final] * interp_fraction)

    interpolated_vectors = np.array([interpolated_positions_x,
                                     interpolated_positions_y,
                                     interpolated_positions_z,
                                     interpolated_masses,
                                     interpolated_metalicities,
                                     interpolated_temperatures,
                                     interpolated_densities]).T

    for los_index in range(completed_sightlines, len(sightline_file)):
        print(f"LOS{los_index}", end = " ")

        los = sightline_file.get_sightline(los_index)
            
        los_quantity_vectors = np.array([los.positions_comoving.to("Mpc")[:, 0],
                                         los.positions_comoving.to("Mpc")[:, 1],
                                         los.positions_comoving.to("Mpc")[:, 2],
                                         los.masses.to("Msun"),
                                         los.metallicities,
                                         los.temperatures.to("K"),
                                         los.densities_comoving.to("g/(cm**3)")]).T
        
        # Find matches
        selected_ids = np.full(len(los), -1, dtype = int)
        for los_part_index in range(len(los)):
            vector_offsets = (np.abs(interpolated_vectors - los_quantity_vectors[los_part_index]) * element_weightings).sum(axis = 1)
            matched_index = vector_offsets.argmin()
            selected_ids[los_part_index] = raw_ids[snap_num_final][matched_index]
        print("duplicates:", len(los) - np.unique(selected_ids).shape[0], "unset:", (selected_ids == -1).sum(), end = " ")
        with h5.File(output_file_location, "a") as file:
            file[output_file_group_name].create_dataset(f"LOS{los_index}", data = selected_ids)
        print("(written to file)")
    print()


part_los_z0.010.hdf5
LOS7 duplicates: 17 unset: 0 (written to file)
LOS8 

KeyboardInterrupt: 